The goal of this notebook is to **Process Google's GTFS Data in order to simplify further analyses**. I will use this data to:
- find out how many trains are operating in the NYC Subway at any given time (284 -- no SIR)
- find the most used stations in the network
- find the ridership per train of each station in the network
- find the trains per hour for each line/station in the network

Data for this project:
- [GTFS Data](https://gtfs.org)
- [Hourly Ridership](https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-Beginning-February-202/wujg-7c2s/about_data)
- [MTA Subway Stations and Complexes](https://catalog.data.gov/dataset/mta-subway-stations-and-complexes)

Some notes about the data and subsequent conclusions:
- the service patterns can change by day of week and time of day (usually late nights) (for example the M only running within Brooklyn during nights and weekends)
- some services include express variants (the 7 and 7 express both count as the 7)
- two services don't run on the weekends: the B and W

# Imports

In [5]:
import pandas as pd
import numpy as np
import geopandas
import seaborn as sns
import datetime
import requests
import warnings
warnings.filterwarnings(action='ignore')

## Functions 

In [6]:
def determine_train_time_intervals(arrival_time, day_of_week: str):
    arrival_time = arrival_time.time()
    train_time_interval = None
    if datetime.time(0, 0) <= arrival_time < datetime.time(6, 30):
        train_time_interval = 'Late Night'
    elif day_of_week in ['Saturday', 'Sunday']:
        train_time_interval = 'Weekend'
    elif datetime.time(6, 30) <= arrival_time < datetime.time(9, 30):
        train_time_interval = 'Rush Hour AM'
    elif datetime.time(9, 30) <= arrival_time < datetime.time(15, 30):
        train_time_interval = 'Midday'
    elif datetime.time(15, 30) <= arrival_time < datetime.time(20, 0):
        train_time_interval = 'Rush Hour PM'
    elif datetime.time(20, 0) <= arrival_time <= datetime.time(23, 59):
        train_time_interval = 'Evening'
    return train_time_interval

def scale_time_intervals_to_hour(df_grouped_at_interval_level: pd.DataFrame):
    for idx, row in df_grouped_at_interval_level.iterrows():
        if row['train_time_interval'] == 'Late Night':
            new_tph = (row['trains_per_hour'] / 3) / 6.5 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
        elif row['train_time_interval'] == 'Weekend':
            new_tph = (row['trains_per_hour'] / 2) / 17.5 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
        elif row['train_time_interval'] == 'Rush Hour AM':
            new_tph = row['trains_per_hour'] / 3 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
        elif row['train_time_interval'] == 'Midday':
            new_tph = row['trains_per_hour'] / 6 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
        elif row['train_time_interval'] == 'Rush Hour PM':
            new_tph = row['trains_per_hour'] / 4.5 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
        elif row['train_time_interval'] == 'Evening':
            new_tph = row['trains_per_hour'] / 4 # multiply by days ratio and hours ratio 
            df_grouped_at_interval_level.at[idx, 'trains_per_hour'] = new_tph
    return df_grouped_at_interval_level

def group_into_day_type(dataframe: pd.DataFrame, column_name:str):    
    day_of_week_list = []
    for x in dataframe[column_name]:
        if 'Weekday' in x or 'L0S1' in x:
            day_of_week_list.append('Weekday')
        elif 'Saturday' in x or 'L0S2' in x:
            day_of_week_list.append('Saturday')
        elif 'Sunday' in x or 'L0S3' in x:
            day_of_week_list.append('Sunday')
    dataframe['day_of_week'] = day_of_week_list
    return dataframe

## Real Time Data

In [7]:
trips_supplemented_df = pd.read_csv("data/google_transit_supplemented/trips.txt")
stops_df_supplemented = pd.read_csv("data/google_transit_supplemented/stops.txt")

## Schedule Data

In [8]:
# holdays and other days when the data would be different than expected
calendar_dates_df = pd.read_csv("data/google_transit/calendar_dates.txt") 
calendar_df = pd.read_csv("data/google_transit/calendar.txt")
routes_df = pd.read_csv("data/google_transit/routes.txt")
stop_times_df = pd.read_csv("data/google_transit/stop_times.txt")
stops_df = pd.read_csv("data/google_transit/stops.txt")
transfers_df = pd.read_csv("data/google_transit/transfers.txt")
trips_df = pd.read_csv("data/google_transit/trips.txt")
# not useful data 
# agency_df = pd.read_csv("google_transit/agency.txt")
shapes_df = pd.read_csv("data/google_transit/shapes.txt")

## Other Data

### Ridership Data
Station ridership for every non-SIR station for every hour in a week (feb 26 - mar 3)

In [9]:
# Only making an api call if we don't already have the hourly station ridership data 
try:
    hourly_station_ridership = pd.read_csv("saved_data/hourly_station_ridership.csv", index_col=0)
except Exception as e:
    copied_url = "https://data.ny.gov/resource/wujg-7c2s.json?$limit=1000000&$where=transit_timestamp >= '2024-2-26T00:00:00' AND transit_timestamp <= '2024-03-03T23:59:59' AND transit_mode = 'subway'&$order=transit_timestamp ASC&$group=transit_timestamp, station_complex_id&$select=transit_timestamp, station_complex_id, sum(ridership) as sum_ridership"
    response = requests.get(copied_url)
    response_data = response.json()
    hourly_station_ridership = pd.DataFrame(response_data)
    hourly_station_ridership['time_as_datetime'] = pd.to_datetime(hourly_station_ridership['transit_timestamp']
                                                              , errors='coerce')
    hourly_station_ridership['station_complex_id'] = hourly_station_ridership['station_complex_id'].astype(float)
    hourly_station_ridership['sum_ridership'] = hourly_station_ridership['sum_ridership'].astype(float)
    # adding the time data 
    hourly_station_ridership['departure_hour'] = hourly_station_ridership['time_as_datetime'].dt.hour
    hourly_station_ridership['departure_day'] = hourly_station_ridership['time_as_datetime'].dt.weekday

### Stations Info

In [10]:
# https://catalog.data.gov/dataset/mta-subway-stations-and-complexes
stations_info_df = pd.read_csv("data/MTA_Subway_Stations_and_Complexes.csv")
stations_info_df = stations_info_df.drop(columns=['ADA', 'ADA Notes'])
# making this easier to explode later
stations_info_df['GTFS_Stop_IDs_Listed'] = [x if ";" not in x else list(x.replace(" ", "").replace(";", ",")
                                                                        .split(",")) 
                                            for x in stations_info_df['GTFS Stop IDs']]

In [11]:
stations_info_df_exploded = stations_info_df.explode("GTFS_Stop_IDs_Listed")

# Adjustments to the GTFS Dfs

## Date Format

In [12]:
# calendar_dates_df['date'].astype('datetime')
calendar_dates_df['date'] = pd.to_datetime(calendar_dates_df['date'], format="%Y%m%d")
calendar_df['start_date'] = pd.to_datetime(calendar_df['start_date'], format="%Y%m%d")
calendar_df['end_date'] = pd.to_datetime(calendar_df['end_date'], format="%Y%m%d")

In [13]:
#making the times within a 24 hour range -- up to 27 
stop_times_df['arrival_time'] = [str(int(x[0:2]) - 24) + x[2:] if int(x[0:2]) >= 24 else x
                          for x in stop_times_df['arrival_time']]
stop_times_df['departure_time']  = [str(int(x[0:2]) - 24) + x[2:] if int(x[0:2]) >= 24 else x
                          for x in stop_times_df['departure_time']]
# making to a datetime
stop_times_df['arrival_time'] = pd.to_datetime(stop_times_df['arrival_time'], format="%H:%M:%S", errors='coerce')
stop_times_df['departure_time'] = pd.to_datetime(stop_times_df['departure_time'], format="%H:%M:%S", errors='coerce')
stop_times_df = group_into_day_type(stop_times_df, 'trip_id')

## Removing Some of the data

In [14]:
# removing lat lon data
# shapes_df.drop(columns=['shape_pt_lat', 'shape_pt_lon'], inplace=True)
stops_df.drop(columns=['stop_lat', 'stop_lon'], inplace=True)
stop_times_df = stop_times_df[~stop_times_df['arrival_time'].isnull()]

## Finding Beginning of Each Route
Using Stop Times df

In [15]:
first_stop_in_trip = stop_times_df[stop_times_df['stop_sequence']==1]
first_stop_in_trip['departure_hour'] = first_stop_in_trip['departure_time'].dt.hour
first_stop_in_trip['route_id'] = [x.split("_")[-1].split('.')[0] 
                                        for x in first_stop_in_trip['trip_id']]
first_stop_in_trip['shape_id'] = [x.split("_")[-1] for x in first_stop_in_trip['trip_id']]
first_stop_in_trip = first_stop_in_trip[~first_stop_in_trip.arrival_time.isnull()]

In [12]:
# Making the data be "per direction" (closer to how an average person would think interpret this)
# the shuttles only have 1 "." -- very close to 50:50 (50.6 ~ 49.4)
directions = []
for x in first_stop_in_trip['shape_id']:
    direction = x.split(".", 1)[1]
    if "S" in direction:
        directions.append("S")
    else:
        directions.append("N")
first_stop_in_trip['direction'] = directions
first_stop_in_trip = first_stop_in_trip[first_stop_in_trip['direction'] == 'S']

### By Day

In [13]:
route_frequency_by_day = pd.DataFrame(first_stop_in_trip.groupby(
                                                ['route_id', 'day_of_week']).size() / 24).reset_index()
route_frequency_by_day.columns = ['route_id', 'day_of_week', 'trains_per_hour']
route_frequency_by_day['headway_seconds'] = 3600 / route_frequency_by_day['trains_per_hour']
route_frequency_by_day['headway_minutes'] = 60 / route_frequency_by_day['trains_per_hour']
route_frequency_by_day = route_frequency_by_day.round(1)

### By Hour

In [14]:
# freq by hour for all routes and all days on schedule 
route_trip_freq_by_hour = first_stop_in_trip.groupby(['route_id', 'day_of_week', 'departure_hour']).sum()
route_trip_freq_by_hour.columns = ['trains_per_hour']
route_trip_freq_by_hour['headway_seconds'] = round(3600 / route_trip_freq_by_hour['trains_per_hour'])
route_trip_freq_by_hour['headway_minutes'] = round(60 / route_trip_freq_by_hour['trains_per_hour'], 1)

### Overall

In [15]:
# avg daily freq all routes 
route_trip_freq_overall_by_hour = route_trip_freq_by_hour['trains_per_hour'].unstack(level=1)
route_trip_freq_overall_by_hour.fillna(value=0, inplace=True) # needed b/c there's no weekend B trains
route_trip_freq_overall_by_hour['Weekday_Adjusted'] = route_trip_freq_overall_by_hour['Weekday'] * 5
route_trip_freq_overall_by_hour['Avg_Hourly_tph'] = (route_trip_freq_overall_by_hour['Weekday_Adjusted'] + \
                                                route_trip_freq_overall_by_hour['Saturday'] + \
                                                route_trip_freq_overall_by_hour['Sunday']) / 7
route_trip_freq_overall_by_hour = route_trip_freq_overall_by_hour.round()

In [16]:
route_trip_freq_overall_by_day = route_trip_freq_overall_by_hour.groupby(['route_id']).sum() / 24
route_trip_freq_overall_by_day.columns = ['Saturday', 'Sunday', 'Weekday'
                                          , 'Weekday_Adjusted', 'Trains_per_Hour_Overall']
# the wait time is half of the mean time between trains
route_trip_freq_overall_by_day['Headway_Minutes'] = 60 / \
                                                        route_trip_freq_overall_by_day['Trains_per_Hour_Overall']
route_trip_freq_overall_by_day = route_trip_freq_overall_by_day.round(1)

### By Train Time Interval

In [17]:
train_time_interval_list = []
for idx, row in first_stop_in_trip.iterrows():
    train_time_interval = determine_train_time_intervals(row['arrival_time'], row['day_of_week'])
    train_time_interval_list.append(train_time_interval)
first_stop_in_trip['train_time_interval'] = train_time_interval_list

In [18]:
first_stop_in_trip_per_interval = pd.DataFrame(first_stop_in_trip.groupby(
                                                ['route_id', 'train_time_interval']
                                                ).size()).reset_index()
first_stop_in_trip_per_interval.columns = ['route_id', 'train_time_interval', 'trains_per_hour']
first_stop_in_trip_per_interval = scale_time_intervals_to_hour(first_stop_in_trip_per_interval)
first_stop_in_trip_per_interval['headway_seconds'] = round(3600 / first_stop_in_trip_per_interval['trains_per_hour'])
first_stop_in_trip_per_interval['headway_minutes'] = round(60 / first_stop_in_trip_per_interval['trains_per_hour'], 1)

### Finding mean trip time by service 

In [19]:
# no trip is longer than 2 hours so I'll drop anything larger than that (the late night trains fuck up the timing)
trip_time_diff = stop_times_df.groupby('trip_id')['departure_time'].agg(np.ptp)
trip_time_diff.mean()

Timedelta('0 days 01:25:06.256322121')

In [20]:
# 97.6% of all trips
valid_trip_times = pd.DataFrame(trip_time_diff[trip_time_diff.values<pd.Timedelta('0 days 02:00:00')]).reset_index()

valid_trip_times['route_id'] = [x.split("_")[-1].split('.')[0] 
                                        for x in valid_trip_times['trip_id']]

In [21]:
avg_trip_time = pd.DataFrame(valid_trip_times.groupby('route_id').mean(numeric_only=False)).reset_index()
avg_trip_time['route_time_seconds'] = [round(x.total_seconds()) for x in avg_trip_time['departure_time']]
avg_trip_time['route_time_minutes'] = [round(x / 60, 1) for x in avg_trip_time['route_time_seconds']]

In [22]:
# the amount of time it takes to reach one end of the route from the other
avg_trip_time_final = avg_trip_time.drop(columns='departure_time')

#### Adding to Daily Frequency Table 

In [23]:
daily_route_trip_freq = route_trip_freq_overall_by_day.merge(avg_trip_time_final, on='route_id')
# doubling the result so that we have ALL trains not just southbound ones
daily_route_trip_freq['Avg_num_trains_running'] = round((daily_route_trip_freq['route_time_minutes'] / 60) * \
                                                        daily_route_trip_freq['Trains_per_Hour_Overall'] * 2, 1)

#### Adding to Hourly

In [24]:
hourly_route_trip_freq = route_trip_freq_by_hour.reset_index().merge(avg_trip_time_final, on='route_id')
# doubling the result so that we have ALL trains not just southbound ones
hourly_route_trip_freq['Avg_num_trains_running'] = (hourly_route_trip_freq['route_time_minutes'] / 60) * \
                                        hourly_route_trip_freq['trains_per_hour'] * 2
hourly_route_trip_freq = hourly_route_trip_freq.round(1)

#### Adding to Train Time Interval

In [25]:
trip_interval_route_freq = first_stop_in_trip_per_interval.merge(avg_trip_time_final, on='route_id')

# doubling the result so that we have ALL trains not just southbound ones
trip_interval_route_freq['Avg_num_trains_running'] = (trip_interval_route_freq['route_time_minutes'] / 60) * \
                                        trip_interval_route_freq['trains_per_hour'] * 2
trip_interval_route_freq = trip_interval_route_freq.round(1)

## Creating the Lines

In [26]:
shapes_df['lat_lon'] = list(zip(round(shapes_df['shape_pt_lat'], 6), round(shapes_df['shape_pt_lon'], 6)))

In [27]:
# less than half as many rows now
shapes_df_unique = shapes_df[shapes_df['shape_id'].str.contains(".S")]

shape_route_dict = dict(zip(trips_df['shape_id'], trips_df['route_id']))
shapes_df_unique['route_id'] = shapes_df_unique['shape_id'].replace(shape_route_dict)

shapes_df_unique['route_id_len'] = [len(x) for x in shapes_df_unique['route_id']]
# removing shapes that aren't in the current dataset 
shapes_df_unique = shapes_df_unique[shapes_df_unique['route_id_len'] <= 2]

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [47]:
shapes_df_unique

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,lat_lon,route_id,route_id_len
1690,1..S03R,0,40.889248,-73.898583,"(40.889248, -73.898583)",1,1
1691,1..S03R,1,40.887195,-73.899616,"(40.887195, -73.899616)",1,1
1692,1..S03R,2,40.886309,-73.900041,"(40.886309, -73.900041)",1,1
1693,1..S03R,3,40.884928,-73.900730,"(40.884928, -73.90073)",1,1
1694,1..S03R,4,40.884667,-73.900870,"(40.884667, -73.90087)",1,1
...,...,...,...,...,...,...,...
169563,SI..S07R,685,40.513696,-74.250493,"(40.513696, -74.250493)",SI,2
169564,SI..S07R,686,40.513579,-74.250706,"(40.513579, -74.250706)",SI,2
169565,SI..S07R,687,40.513458,-74.250917,"(40.513458, -74.250917)",SI,2
169566,SI..S07R,688,40.513334,-74.251124,"(40.513334, -74.251124)",SI,2


## Length of Cars for Each Line 

In [101]:
# a dictionary showing the length of each car for each line 
# source: https://en.wikipedia.org/wiki/New_York_City_Subway_rolling_stock
typical_length_of_cars = {'A': 15.5, 'B1': 18.4, 'B2':23}
typical_width_of_cars = {'A': 2.7, 'B': 3.1}

In [102]:
unique_lines = list(first_stop_in_trip.route_id.unique())
division_split_dict = dict(zip(unique_lines
                             , [None for x in range(len(unique_lines))]))
for key, value in division_split_dict.items():
    try:
        if key == 'GS':
            division_split_dict[key] = 'A'
        elif int(key):
            division_split_dict[key] = 'A'
    except Exception as e:
        division_split_dict[key] = 'B'

In [104]:
train_area_df = pd.DataFrame.from_dict(division_split_dict, orient='index', columns=['division']).reset_index()
train_area_df.columns = ['route_id', 'division']

In [105]:
# giving all values a car length
# we'll multiply their value by the number of cars to get the final length
car_length_dict = division_split_dict.copy()
for key, value in car_length_dict.items():
    if value == 'A':
        car_length_dict[key] = typical_length_of_cars[value]
    elif key in ['J', 'L', 'M', 'Z']:
        car_length_dict[key] = typical_length_of_cars['B1']
    else:
        car_length_dict[key] = typical_length_of_cars['B2']
train_area_df['car_length'] = car_length_dict.values()
train_area_df['car_width'] = [typical_width_of_cars[x] for x in train_area_df['division']]

In [107]:
cars_per_train_dict = dict(zip(unique_lines, [None for x in range(len(unique_lines))]))
# B1 division
cars_per_train_dict['J'] = 8
cars_per_train_dict['M'] = 8
cars_per_train_dict['L'] = 8
# Shuttles
cars_per_train_dict['FS'] = 2
cars_per_train_dict['H'] = 4
cars_per_train_dict['GS'] = 6
# Other
cars_per_train_dict['SI'] = 4
cars_per_train_dict['G'] = 5
cars_per_train_dict['7'] = 11

In [109]:
train_area_df['number_of_cars'] = pd.DataFrame.from_dict(cars_per_train_dict, orient='index')
for idx in range(len(train_area_df)):
    value = train_area_df['number_of_cars'][idx]
    if np.isnan(value):
        if train_area_df['division'][idx] == 'A':
            value = 10
        elif train_area_df['division'][idx] == 'B':
            value = 8
    train_area_df['number_of_cars'][idx] = value

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [110]:
train_area_df['train_length'] = train_area_df['car_length'] * train_area_df['number_of_cars']
train_area_df['trainset_area'] = train_area_df['train_length'] * train_area_df['car_width']
train_area_dict = dict(zip(train_area_df.index, train_area_df['trainset_area']))
train_area_df['capacity_per_car'] = [180 if train_area_df['division'][x] == 'A' else 240 
                                     for x in range(train_area_df.shape[0])]
# B division capacity is around 240 people per car (https://en.wikipedia.org/wiki/R160_(New_York_City_Subway_car)#cite_note-5)
# A division capacity is around 180 people per car (https://en.wikipedia.org/wiki/R142_(New_York_City_Subway_car))
train_area_df['capacity'] = [int(train_area_df['number_of_cars'][x] * train_area_df['capacity_per_car'][x])
                             for x in range(train_area_df.shape[0])]

## Station (Complex) Data

### Grouping at Station Complex Level

In [115]:
# I can find the lines that connect each station
## I would need to further combine them to equal the service pattern
parent_stops_df = stops_df_supplemented[stops_df_supplemented['parent_station'].isnull()==True]
parent_stops_df['lat_lon'] = list(zip(round(parent_stops_df['stop_lat'], 6)
                                      , round(parent_stops_df['stop_lon'], 6)))

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [116]:
stop_times_df['route_id'] = [x.split("_")[-1].split('.')[0] 
                                        for x in stop_times_df['trip_id']]
stop_times_df['parent_stop_id'] = [x[0:-1] for x in stop_times_df['stop_id']]

In [117]:
# group each segment (between stations)
# identify which lines use that segment
# add up the number of trains that use that segment 
# scale the width of that segment to the number of trains per hour 

In [118]:
train_time_interval_list = []
for idx, row in stop_times_df.iterrows():
    train_time_interval = determine_train_time_intervals(row['arrival_time'], row['day_of_week'])
    train_time_interval_list.append(train_time_interval)
stop_times_df['train_time_interval'] = train_time_interval_list

In [119]:
stop_times_df['trainset_area'] = stop_times_df['route_id'].replace(train_area_dict)

In [123]:
# trains per hour during weekdays 
# this includes both directions so I'm dividing by 2 to make it "per direction"
trains_per_weekday_hour_by_station = round(stop_times_df[stop_times_df['day_of_week'] == 'Weekday'
                                                  ].groupby('parent_stop_id').count() / 24 / 2, 2)[['trip_id']]
trains_per_weekday_hour_by_station.reset_index(inplace=True)

In [125]:
trains_per_weekday_hour_by_station

,parent_stop_id,trip_id
0,101,9.00
1,103,9.33
2,104,9.33
3,106,9.33
4,107,9.42
...,...,...
491,S27,2.52
492,S28,2.52
493,S29,2.52
494,S30,2.52


In [126]:
train_area_df

,route_id,division,car_length,car_width,number_of_cars,train_length,trainset_area,capacity_per_car,capacity
0,1,A,15.5,2.7,10.0,155.0,418.50,180,1800
1,2,A,15.5,2.7,10.0,155.0,418.50,180,1800
2,3,A,15.5,2.7,10.0,155.0,418.50,180,1800
3,4,A,15.5,2.7,10.0,155.0,418.50,180,1800
4,5,A,15.5,2.7,10.0,155.0,418.50,180,1800
5,6,A,15.5,2.7,10.0,155.0,418.50,180,1800
6,GS,A,15.5,2.7,10.0,155.0,418.50,180,1800
7,A,B,23.0,3.1,8.0,184.0,570.40,240,1920
8,B,B,23.0,3.1,8.0,184.0,570.40,240,1920
9,C,B,23.0,3.1,8.0,184.0,570.40,240,1920


In [121]:
# train area per hour during weekdays per direction
train_area_per_weekday_hour_by_station = round(stop_times_df[stop_times_df['day_of_week'] == 'Weekday'
                                                  ].groupby('parent_stop_id').sum() / 24 / 2, 2)[['trainset_area']]
train_area_per_weekday_hour_by_station.reset_index(inplace=True)

KeyError: "None of [Index(['trainset_area'], dtype='object')] are in the [columns]"

In [ ]:
train_info_per_weekday_hour_by_station = pd.merge(trains_per_weekday_hour_by_station
                                                  , train_area_per_weekday_hour_by_station)
train_info_per_weekday_hour_by_station.columns = ['parent_stop_id', 'trip_id', 'hourly_trainset_area']

In [ ]:
trains_per_weekday_hour_by_complex = train_info_per_weekday_hour_by_station.merge(
                                right=stations_info_df_exploded[['Complex ID', 'GTFS_Stop_IDs_Listed', ]]
                                , how='outer', left_on='parent_stop_id', right_on='GTFS_Stop_IDs_Listed')
trains_per_weekday_hour_by_complex = trains_per_weekday_hour_by_complex.groupby('Complex ID').sum().reset_index()

In [ ]:
trains_per_weekday_hour_by_complex_all_info = trains_per_weekday_hour_by_complex.merge(
                                                            stations_info_df_exploded.drop(columns="Is Complex")
                                                              , how='outer')
trains_per_weekday_hour_by_complex_all_info.drop_duplicates(subset='Complex ID', inplace=True)

In [ ]:
trains_per_weekday_hour_by_complex_all_info.columns = ['Complex ID', 'tph', 'hourly_trainset_area'
                                                       , 'Number Of Stations In Complex', 'Stop Name',
                                               'Display Name', 'Constituent Station Names', 'Station IDs',
                                               'GTFS Stop IDs', 'Borough', 'Daytime Routes', 'Structure Type',
                                               'Latitude', 'Longitude', 'GTFS_Stop_IDs_Listed']

In [ ]:
# mean wait time should be tph / number of stations in complex
# (if 80 trains come in an hour the wait time isnt really 45 sec)
## -- wait time only makes sense at the station level because of this 
trains_per_weekday_hour_by_station['mean_wait_time_min'] = round(60 / \
                                                            trains_per_weekday_hour_by_station['trip_id'], 1)

### Adding Ridership to each Station/Station Complex 

In [ ]:
hourly_station_ridership_weekday = hourly_station_ridership[hourly_station_ridership['departure_day'] < 5]

In [ ]:
# grouping to the weekday avg. hourly ridership by complex
hourly_station_ridership_weekday_grouped = hourly_station_ridership_weekday.groupby(
                                        ['station_complex_id', 'departure_hour']).sum().reset_index()
hourly_station_ridership_weekday_grouped.drop(columns='departure_day', inplace=True)
# making it to be the average weekday
hourly_station_ridership_weekday_grouped['avg_hourly_ridership'] = \
                                                    hourly_station_ridership_weekday_grouped['sum_ridership'] / 5
hourly_station_ridership_weekday_grouped.drop(columns='sum_ridership', inplace=True)

In [ ]:
daily_weekday_ridership = hourly_station_ridership_weekday_grouped.groupby('station_complex_id').sum().reset_index()
daily_weekday_ridership.drop(columns='departure_hour', inplace=True)
daily_weekday_ridership.columns = ['station_complex_id', 'avg_daily_ridership']

### Combining the two

In [ ]:
station_complex_full_info = trains_per_weekday_hour_by_complex_all_info.merge(daily_weekday_ridership
                                                , left_on='Complex ID'
                                                 , right_on='station_complex_id', how='inner')

In [ ]:
# getting the riders per hour per train
station_complex_full_info['avg_hourly_ridership'] = round((station_complex_full_info['avg_daily_ridership']) 
                                                          / 24, 1)
# getting the riders per hour per train direction (tph is only in S direction)
station_complex_full_info['hourly_ridership_per_train_direction'] = round((station_complex_full_info['avg_hourly_ridership'])\
                                                    / (station_complex_full_info['tph'] * 2), 1)
# station_complex_full_info['hourly_ridership_per_sq_meter'] = round(
#                                                     (station_complex_full_info['avg_hourly_ridership'])\
#                                                 / (station_complex_full_info['hourly_trainset_area'] * 2), 3)

In [ ]:
station_complex_full_info.columns = ['Complex_ID', 'tph', 'hourly_trainset_area'
                                     , 'Number Of Stations In Complex', 'Stop Name', 'Display Name'
                                     , 'Constituent Station Names', 'Station IDs','GTFS Stop IDs', 'Borough'
                                     , 'Daytime Routes', 'Structure Type', 'Latitude', 'Longitude'
                                     , 'GTFS_Stop_IDs_Listed', 'station_complex_id', 'avg_daily_ridership'
                                     , 'avg_hourly_ridership', 'hourly_ridership_per_train_direction']

### Finding Station Frequency by Time Interval
how much service does each station get by time interval?

In [ ]:
# sum of stations by train time interval
# it also needs to be scaled because late night has 3 days whereas rush hour only covers 1 day (when it covers 5)
frequency_by_stop_interval = stop_times_df.groupby(
                                ['parent_stop_id', 'train_time_interval']).count()[['arrival_time']]
frequency_by_stop_interval.columns = ['trains_per_hour']
frequency_by_stop_interval = frequency_by_stop_interval.reset_index()
# standardizing the intervals (weekend covers a much larger time span and late night covers 3 days)
frequency_by_stop_interval = scale_time_intervals_to_hour(frequency_by_stop_interval)

# Saving the Transformed Data

In [ ]:
first_stop_in_trip.to_csv("saved_data/first_stop_in_trip.csv")
hourly_route_trip_freq.to_csv("saved_data/hourly_route_trip_freq.csv")
frequency_by_stop_interval.to_csv("saved_data/frequency_by_stop_interval.csv")
daily_route_trip_freq.to_csv("saved_data/daily_route_trip_freq.csv")
route_frequency_by_day.to_csv("saved_data/route_frequency_by_day.csv") # AKA first_stop_in_trip_per_day
trip_interval_route_freq.to_csv("saved_data/trip_interval_route_freq.csv")
train_area_df.to_csv("saved_data/length_of_each_train.csv")
hourly_station_ridership.to_csv("saved_data/hourly_station_ridership.csv")
station_complex_full_info.to_csv("saved_data/station_complex_full_info.csv")